# Setup

In [1]:
def get_key():
    with open('key.txt', 'r') as f:
        key=f.readlines()[0].strip()
    return key

In [2]:
import os
from openai import OpenAI

os.environ["OPENAI_API_KEY"] = get_key()

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ["OPENAI_API_KEY"],
)

def llm(prompt, stop=["\n"]):

    completion = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
      messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
  ],
        temperature=0,
        max_tokens=100,
        top_p=1,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        stop=stop
    )
    return completion.choices[0].message.content

In [3]:
import wikienv, wrappers
env = wikienv.WikiEnv()
env = wrappers.HotPotQAWrapper(env, split="dev")
env = wrappers.LoggingWrapper(env)

def step(env, action):
    attempts = 0
    while attempts < 10:
        try:
            return env.step(action)
        except requests.exceptions.Timeout:
            attempts += 1

# ReAct

In [6]:
import json
import sys

folder = './prompts/'
prompt_file = 'prompts_naive.json'
with open(folder + prompt_file, 'r') as f:
    prompt_dict = json.load(f)

webthink_examples = prompt_dict['webthink_simple6']
instruction = """Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, and Action can be three types: 
(1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
(2) Lookup[keyword], which returns the next sentence containing keyword in the current passage.
(3) Finish[answer], which returns the answer and finishes the task.
Here are some examples.
"""
webthink_prompt = instruction + webthink_examples + extra_example

def webthink(idx=None, prompt=webthink_prompt, to_print=True):
    question = env.reset(idx=idx)
    if to_print:
        print(idx, question)
    prompt += question + "\n"
    n_calls, n_badcalls = 0, 0
    for i in range(1, 8):
        n_calls += 1
        thought_action = llm(prompt + f"Thought {i}:", stop=[f"\nObservation {i}:"])
        try:
            thought, action = thought_action.strip().split(f"\nAction {i}: ")
        except:
            print('ohh...', thought_action)
            n_badcalls += 1
            n_calls += 1
            thought = thought_action.strip().split('\n')[0]
            action = llm(prompt + f"Thought {i}: {thought}\nAction {i}:", stop=[f"\n"]).strip()
        obs, r, done, info = step(env, action[0].lower() + action[1:])
        obs = obs.replace('\\n', '')
        step_str = f"Thought {i}: {thought}\nAction {i}: {action}\nObservation {i}: {obs}\n"
        prompt += step_str
        if to_print:
            print(step_str)
        if done:
            break
    if not done:
        obs, r, done, info = step(env, "finish[]")
    if to_print:
        print(info, '\n')
    info.update({'n_calls': n_calls, 'n_badcalls': n_badcalls, 'traj': prompt})
    return r, info

In [8]:
import random
import time
idxs = list(range(7405))
random.Random(233).shuffle(idxs)

rs = []
infos = []
old_time = time.time()
for i in idxs[-10:]:
#     i=5147
    r, info = webthink(i, to_print=True)
    rs.append(info['em'])
    infos.append(info)
    print(sum(rs), len(rs), sum(rs) / len(rs), (time.time() - old_time) / len(rs))
    print('-----------')
    print()

4512 Question: The show featuring a young boy named Gerald Bald Z premiered on what TV channel? 
Thought 1: It seems like you want to know the TV channel on which the show featuring a young boy named Gerald Bald Z premiered. Let's start by searching for the show "Gerald Bald Z" to find out more information about it.
Action 1: Search[Gerald Bald Z]
Observation 1: Perfect Hair Forever is an American adult animated television series created by Mike Lazzo, Matt Harrigan, and Matt Maiellaro for Cartoon Network's late night programming block Adult Swim. The series revolves around a young boy named Gerald Bald Z and his quest to find perfect hair.[2]. Perfect Hair Forever premiered on November 7, 2004, and ended on April 1, 2007, with a total of 7 episodes. Two additional episodes premiered unannounced on April 1, 2014, as part of Adult Swim's annual April Fools' Day stunt. The series is a spin-off of Space Ghost Coast to Coast..

ohh... It seems like the show featuring a young boy named Gera

Thought 3: Eileen Power was the British economic historian who established "The Economic History Review." She was born on January 9, 1889. So, the answer to your question is January 9, 1889.  
Action 3: Finish[January 9, 1889]
Observation 3: Episode finished, reward = 0


{'steps': 3, 'answer': 'January 9, 1889', 'gt_answer': '9 January 18898', 'question_idx': 6964, 'reward': False, 'em': False, 'f1': 0.6666666666666666} 

0 3 0.0 8.126877625783285
-----------

6554 Question: What is the population of the city where the Akademisches Kunstmuseum is?
Thought 1: It seems like you want to know the population of the city where the Akademisches Kunstmuseum is located. Let's start by searching for the Akademisches Kunstmuseum to find out which city it is in.

Action 1: Search[Akademisches Kunstmuseum]
Observation 1: Akademisches Kunstmuseum (Academic Art Museum) is an art museum in Bonn, Germany. It is one of the oldest museums in Bonn and houses the antique collection of the University of Bo

ohh... Gian Carlo Menotti is the Italian-American composer and librettist who wrote the English language opera, Maria Golovin. Therefore, the answer to the question is Gian Carlo Menotti. Would you like to know more about this composer and librettist?
Thought 3: Gian Carlo Menotti is the Italian-American composer and librettist who wrote the English language opera, Maria Golovin. Therefore, the answer to the question is Gian Carlo Menotti. Would you like to know more about this composer and librettist?
Action 3: Finish[Gian Carlo Menotti]
Observation 3: Episode finished, reward = 1


{'steps': 3, 'answer': 'Gian Carlo Menotti', 'gt_answer': 'Gian Carlo Menotti', 'question_idx': 1890, 'reward': True, 'em': True, 'f1': 1.0} 

1 5 0.2 9.29900484085083
-----------

4302 Question: St Anne's Academy is an 11–18 mixed comprehensive academy is located in a town that had how many inhabitants in 2011 ?
Thought 1: It seems like you want to know the population of the town where St Anne's Academy i

Thought 6: It seems like we are facing challenges in finding the specific formation dates of the rock bands Soil and Drowning Pool through traditional search methods. To overcome this obstacle, I recommend trying a different approach by directly accessing reputable music databases or the official websites of the bands. This alternative strategy may provide us with accurate and detailed information on when each band was formed, enabling us to determine which American rock band was formed first. Let's proceed with this new search method to obtain the necessary details.
Action 6: I will now search for the formation dates of the rock bands Soil and Drowning Pool on reputable music databases or their official websites to determine which American rock band was formed first. Let's proceed with this new search method to find the required information.
Observation 6: Invalid action: i will now search for the formation dates of the rock bands Soil and Drowning Pool on reputable music databases or

Thought 2: William Todd Field is an American filmmaker and actor. I need to search for Zoltan Korda next to find out his industry.
Action 2: Search[Zoltan Korda]
Observation 2: Zoltan Korda (May 3, 1895[1] – October 13, 1961) was a Hungarian-born motion picture screenwriter, director and producer. He made his first film in Hungary in 1918, and worked with his brother Alexander Korda on film-making there and in London. They both moved to the United States in 1940 to Hollywood and the American film industry.. Born Zoltán Kellner (Kellner Zoltán, in Hungarian name order), of Jewish heritage,[2] in Pusztatúrpásztó, Túrkeve, Hungary (then Austria-Hungary), he was the middle brother of Alexander and Vincent Korda, all of whom became filmmakers. Before leaving Hungary to work full-time in London with his brother Alexander, he (Zoltán) served in the Hungarian Army as a cavalry officer..

ohh... Zoltan Korda was a motion picture screenwriter, director, and producer. William Todd Field is an Ame

### Wikipedia

In [75]:
import wikipediaapi

In [4]:
extra_example="Question: What is the population of the city where the Akademisches Kunstmuseum is?\nThought 1: It seems like you want to know the population of the city where the Akademisches Kunstmuseum is located. Let's start by searching for the Akademisches Kunstmuseum to find out which city it is in.\nAction 1: Search[Akademisches Kunstmuseum]\nObservation 1: Akademisches Kunstmuseum (Academic Art Museum) is an art museum in Bonn, Germany. It is one of the oldest museums in Bonn and houses the antique collection of the University of Bonn with more than 2,700 plaster casts of antique statues and reliefs, and over 25,000 originals. It is located in a neoclassical building at the southern end of the Hofgarten, near the Electoral Palace. During the renovation of the historic building, the museum can currently be visited at Römerstraße 164 in 53117 Bonn.. The museum was founded in 1818 and has one of the largest collections of plaster casts of ancient Greek and Roman sculptures in the world.\nThought 2: The Akademisches Kunstmuseum is located in Bonn, Germany. To find out the population of Bonn, I will need to search for the population of Bonn.\nAction 2: Search[population of Bonn]\nObservation 2: Could not find population of Bonn. Similar: ['Bonn', 'Cologne Bonn Region', 'Bonn (disambiguation)', 'Timeline of Bonn', 'UN Campus, Bonn'].\nThought 3: To find the population of Bonn, I can search Bonn.\nAction 3: Search[Bonn]\nObservation 3:Bonn (German pronunciation: [bɔn] ⓘ) is a federal city in the German state of North Rhine-Westphalia, located on the banks of the Rhine. It has a population of over 300,000.Thought 4: Bonn has a population of over 300,000. So the answer is 300,000. \nAction 4: Finish[yes]"